
# **Gestión de datos y datos digitales**
### **Autor: Ferran Carrascosa Mallafrè**

---
---

<!-- script html for image -->

<center>


<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c9/Star_Wars%2C_%C3%A9pisode_III_-_La_Revanche_des_Sith_logo.jpg/220px-Star_Wars%2C_%C3%A9pisode_III_-_La_Revanche_des_Sith_logo.jpg"  width="500" height="150"/>


<br>

Fuente de la imagen: [https://es.wikipedia.org](https://es.wikipedia.org/wiki/Star_Wars:_Episodio_III_-_La_venganza_de_los_Sith)

</center>

<br>

# **Índice**

---
---

> [Gestión de datos y datos digitales](#scrollTo=eJfiC-K0VViA&uniqifier=1)
<br>
>>
>> [Preparación del Entorno](#scrollTo=UkU8akIVtlbn&line=1&uniqifier=1) \\
>>   \\
>> [1.3. Google Analytics](#scrollTo=U3oTMspTVVjE&uniqifier=1) \\
>>
>>> [1.3.1. Google developer console](#scrollTo=Hd88I7fxVVju&uniqifier=1) \\
>>>
>>> [1.3.2. Configuración en Google Analytics 4](#scrollTo=yxlPi4fjjYGb&uniqifier=1) \\
>>>
>>> [1.3.3. Acceso a Google Analytics des de Python](#scrollTo=uNp0KfRlVVkP&uniqifier=1)
>>
>> [Enlaces de interés](#scrollTo=C-AMazPq4Htt&uniqifier=1)

# Preparación del entorno

Cargamos las librerías.

In [ ]:
# Instala las librerías necesarias
# !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib pandas
# !pip install google-analytics-data pandas google-auth


In [71]:
import pandas as pd
import numpy as np
import json

from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


# 1.3. Google Analytics


Una de las primeras tareas para crear un nuevo website es configurar Google Analytics. Después del tiempo y esfuerzo invertido en construir una web, habitualmente se quiere medir y analizar el comportamiento de los usuarios.

Las preguntas más frecuentes son:

- ¿Como esta funcionando mis contenidos?
- ¿Qué categorías de productos son los más populares?
- ¿Cuantas conversiones ha realizado mi campaña de Facebook?
- ¿Los usuarios ven el último video que he producido?
- ¿Que comparten los usuarios en las redes sociales?
- ¿Las personas abandonan la pagina durante el proceso de pago?


El [46,4 % de todos los sitios web](https://w3techs.com/technologies/details/ta-googleanalytics) utiliza Google Analytics y esto supone el [81.2% del mercado de herramientas de analisis web](https://w3techs.com/technologies/overview/traffic_analysis).

El objetivo es poder tomar decisiones basados en datos. Las decisiones pueden ser, por ejemplo, el tipo de contenidos invertir, o qué presupuesto puedes gastar en campañas de anuncios.

El objetivo de este módulo es aprender a conectar con la API de Datos de GA4 para extraer información a través de Python y convertirla en DataFrames de Pandas.

> **Importante**: Obviamente, para poder practicar con el contenido de este módulo necesitas tu propia cuenta de Google y una propiedad de Google Analytics 4.


## 1.3.1. Google developer console

Para acceder puedes acceder al siguiente enlace web: https://console.developers.google.com/. 

Una vez dentro, debes crear un proyecto nuevo. Esto lo puedes hacer en la pestaña que encontrarás en la parte superior izquierda. Clicas en la pestaña y aprietas la opción de **NUEVO PROYECTO**.

Tienes que dar un nombre al proyecto. Por ejemplo "GA master IL3" y le das al botón de crear.

### Habilita la API de GA4:

En el menú de la izquierda, ve a "Biblioteca".

Busca y habilita la "Google Analytics Data API".

### Crea una Cuenta de Servicio:

- En el menú de la izquierda, ve a "Credenciales"
  - Clica en "Crear Credenciales" (superior) y selecciona "Cuenta de Servicio".
  - Dale un nombre (ej. ga4-il3-test) y clica en "Crear y continuar".
  - En "Función", asigna el rol de "Administrador" para simplificar y clica en "Continuar" y luego en "Listo".


> **Importante**: Revisa que se ha creado la Cuenta de servicio que deberia llevar el siguiente nombre (según lo hayas configurado):   ga-il3-test@ga-master-il3.iam.gserviceaccount.com. Este correo, lo necesitaras mas adelante.

### Crea una Clave JSON:

En la lista de credenciales, busca la cuenta de servicio que acabas de crear (tendrá un email como ga-il3-test@ga-master-il3.iam.gserviceaccount.com ).

- Clica sobre ella.
- Ve a la pestaña "CLAVES".
- Clica en "AGREGAR CLAVE" > "Crear clave nueva".
- Deja el tipo de clave como JSON y clica en "CREAR".
- Se descargará automáticamente un archivo .json en tu ordenador, en mi caso se llama: "ga-master-il3-xxxxxxxxxxx.json" (las  x son un numero de serie, no es importante). Este archivo es tu credencial, guárdalo bien.



## 1.3.2. Configuración en Google Analytics 4

Si no las tienes, debes crear una **cuenta** y un **propiedad** desde https://analytics.google.com/ (sigue los formularios, es simple). En mi caso he creado un acuenta "master IL3" y una propiedad. Vemos este útlimo paso.

### Crear la Propiedad de GA4

- Da un nombre a la propiedad, en mi caso "Test IL3 2025". Tambien una zona y moneda. Siguiente.
- Selecciona una categoria (Trabajo y educación, p.e.) y una tamaño de empresa (Pequeña). Siguiente.
- Seleccina uno o dos objetivos. Crear.
- En recogida de datos, debes selccioanr una plataforma. En mi caso Web:
  - Debes aportar una url, por ejemplo, abre un blog en [ blogger](https://www.blogger.com/) e introduce la url. En mi caso testil32025.blogspot.com. Finalmente da un nombre al flujo de datos.
  - Si no se abre automáticamente, clica en el nuevo flujo y cuadro de "Consulta las instrucciones de la etiqueta" > "Instala manualmente". Deberia aparecer las instrucciones para añadir el script de monitorización a tu web:

Insertar un script de seguimiento en la cabecera o sección <head> de tu página web.

```html
<!-- Google tag (gtag.js) -->
<script async src="https://www.googletagmanager.com/gtag/js?id=G-XXXXXXXXXX"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'G-XXXXXXXXXX');
</script>
```

En el caso de blogger debes ir a "Configuración" > En "Opciones básicas", verás el apartado: "Identificador de propiedad de Google Analytics".  > Clicar > copiar el tag G-XXXXXXXXXX > Guardar

> **Importante**: Ahora, el tráfico web ya quedará registrado. Puedes comporbarlo accediendo a tu web/blogger y verificar el tráfico de accesos en los informes en Tiempo real de Google analytics (tarda unos segundos).

El siguiente paso consiste en dar acceso a la cuenta de servicio para que pueda leer los datos de nuestra propiedad de GA4.

### Obtener el ID de Propiedad de GA4

- Entra en tu cuenta de Google Analytics: https://analytics.google.com/
- Ve a Administrar (el icono de la rueda dentada ⚙️ en la esquina inferior izquierda).
- Asegúrate de tener seleccionada la cuenta y la propiedad de GA4 correctas. Verás dos secciones: "Cuenta" y "Propiedad".
- En la secciñon "Propiedad", clica en "Configuración de la propiedad".
- En la parte superior derecha, verás el ID DE PROPIEDAD. Es un número largo (ej. 345678910). NO es el ID de medición que empieza por "G-".
- Copia este número. Lo necesitarás en el script de Python.

### Asignar Permisos a la Cuenta de Servicio

- En la misma columna de "Propiedad", clica en "Gestión de accesos a la propiedad".
- Clica en el botón azul con el signo + en la esquina superior derecha y selecciona "Añadir usuarios".
- En el campo "Direcciones de correo electrónico", pega el email de la cuenta de servicio que creaste en la Google Console (lo encontrarás en el archivo .json, en el campo client_email).
- Desmarca la opción "Notificar a los usuarios nuevos por correo electrónico".
- En "Función", asigna el rol de "Lector" (Viewer). Es suficiente para leer datos.
- Clica en "Añadir".

## 1.3.3. Acceso a Google Analytics des de Python

Para acceder a los datos de Google Analytics, vas a necesitar las credenciales que has generado en los apartado anteriores (arxivo: "ga-master-il3-xxxxxxxxxxx.json").

Sube este archivo a Colab (o en tu carpeta local de Jupyter Notebook) a través del icono de Files (menú izquierda) > Icono de "Upload Session Storage" > busca el archivo "ga-master-il3-xxxxxxxxxxx.json" > "Aceptar”

Cargamos las librerías.


In [ ]:
import pandas as pd
import numpy as np

from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


Leemos las credenciales.


In [ ]:
# Reemplaza 'tu_archivo.json' con el nombre de tu archivo de credenciales
KEY_FILE_LOCATION = 'ga-master-il3-XXXXXXXXXXX.json'    # modifica la ruta y nombre fichero en caso necesario

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']



Se define la funciona de obtención de credenciales de Google Analytics.


In [64]:
def get_analytics_service():
    """Inicializa un servicio para la API de datos de GA4."""
    credentials = service_account.Credentials.from_service_account_file(
        KEY_FILE_LOCATION, scopes=SCOPES)
    
    # Construimos el servicio para la API de Datos de GA4 (v1beta)
    service = build('analyticsdata', 'v1beta', credentials=credentials)
    
    return service

Creamos las funciones para pedir el informe y para convertir la respuesta en un DataFrame.

In [65]:
def run_ga4_report(service, property_id, dimensions, metrics, start_date, end_date):
    """
    Ejecuta un informe en la API de datos de GA4 y devuelve la respuesta.
    """
    try:
        # Crea el cuerpo de la solicitud
        request_body = {
            "dimensions": [{"name": dim} for dim in dimensions],
            "metrics": [{"name": met} for met in metrics],
            "dateRanges": [{"startDate": start_date, "endDate": end_date}],
        }

        # Ejecuta la solicitud
        response = service.properties().runReport(
            property=f"properties/{property_id}",
            body=request_body
        ).execute()
        
        return response
    except HttpError as error:
        print(f"Ha ocurrido un error: {error}")
        return None


def parse_ga4_response_to_df(response):
    """
    Convierte la respuesta JSON de la API de GA4 en un DataFrame de pandas.
    """
    if response is None or not response.get('rows'):
        print("La respuesta está vacía o no contiene filas de datos.")
        return pd.DataFrame()

    # Extraer las cabeceras
    dim_headers = [h['name'] for h in response['dimensionHeaders']]
    metric_headers = [h['name'] for h in response['metricHeaders']]
    headers = dim_headers + metric_headers

    # Extraer las filas de datos
    rows = []
    for row in response.get('rows', []):
        row_values = [dim['value'] for dim in row['dimensionValues']]
        row_values.extend([met['value'] for met in row['metricValues']])
        rows.append(row_values)
    
    return pd.DataFrame(rows, columns=headers)


¡Llegó el momento de usarlo!

Copia tu ID de Propiedad en la siguiente variable.

> **Importante**: Los datos pueden tardar hasta 24-48 horas en aparecer en la API después de configurar tu sitio (e mi caso unsos 15 minutos). Asegúrate de que tu web está generando tráfico.

In [72]:
# --- CONFIGURACIÓN ---
# Reemplaza con tu ID de Propiedad de GA4
PROPERTY_ID = '493423124'  # Ejemplo: '345678910'


# Inicializamos el servicio
analytics_service = get_analytics_service()

# Definimos las dimensiones y métricas para nuestro primer informe
# Queremos ver las páginas más vistas
dimensions = ['pagePath']
metrics = ['activeUsers', 'screenPageViews']

# Realizamos la petición para los últimos 30 días
response = run_ga4_report(analytics_service, PROPERTY_ID, dimensions, metrics, '30daysAgo', 'today')

# Convertimos la respuesta en un DataFrame
df_pageviews = parse_ga4_response_to_df(response)

print("--- Páginas más vistas los últimos 30 días ---")
print(df_pageviews.head())

--- Páginas más vistas los últimos 30 días ---
  pagePath activeUsers screenPageViews
0        /           4               6



Si todo ha ido bien, creamos las funciones de obtención del informe y parseado de los resultados.


Puedes probar con otras dimensiones y métricas. Los nombres han cambiado respecto a Universal Analytics (ahora no llevan el prefijo ga:).

- browser (Navegador)
- sessionSourceMedium (Fuente y medio de la sesión)
- country (País)
- sessions (Sesiones)

In [73]:
# --- SEGUNDO EJEMPLO: Sesiones por fuente/medio y navegador ---

dimensions_2 = ['sessionSourceMedium', 'browser']
metrics_2 = ['sessions']
start_date_2 = '2025-05-01'
end_date_2 = '2025-06-14'

response_2 = run_ga4_report(analytics_service, PROPERTY_ID, dimensions_2, metrics_2, start_date_2, end_date_2)
df_source_browser = parse_ga4_response_to_df(response_2)

print(f"\n--- Sesiones por Fuente/Medio y Navegador ({start_date_2} a {end_date_2}) ---")
print(df_source_browser.head())


--- Sesiones por Fuente/Medio y Navegador (2025-05-01 a 2025-06-14) ---
  sessionSourceMedium browser sessions
0           (not set)  Chrome        3
1   (direct) / (none)  Chrome        2


Para explorar todas las dimensiones y métricas disponibles en la API de datos de GA4, consulta la documentación oficial:

- [GA4 Dimensions & Metrics Explorer](https://developers.google.com/analytics/devguides/reporting/data/v1/api-schema)

Enlaces de Interés (Actualizados para GA4):

- [Documentación de la API de Datos de GA4](https://developers.google.com/analytics/devguides/reporting/data/v1)
- [Guía de Migración de UA a GA4 para desarrolladores](https://developers.google.com/analytics/devguides/migration/api/reporting-ua-to-ga4)
- [Ejemplos de código Python para la API de GA4](https://www.google.com/search?q=https://github.com/googleanalytics/python-analytics-data)